In [3]:
from Bio import SeqIO
import pandas as pd


for marker in ['16SrRNA','ITS1']:
    
    outdir = 'picrust/%s' % marker
    
    !mkdir -p $outdir
    
    qza = 'dada/%s/rep-seqs.qza' % marker
    a = !unzip $qza
    digest = a[1].split('/')[0].replace('  inflating: ','')
    inf = digest + '/data/dna-sequences.fasta'
    repseqs = SeqIO.to_dict(SeqIO.parse(inf,'fasta'))
    !rm -r $digest 

    #get biom
    qza = 'diversity/%s/rarefied_table.qza' % marker
    a = !unzip $qza
    digest = a[1].split('/')[0].replace('  inflating: ','')
    inf = digest + '/data/feature-table.biom'
    biom_file = 'picrust/%s/table.biom' % marker
    a = !cp $inf $biom_file

    # get fasta
    outf = 'temp.%s' % digest
    a = !biom convert -i $inf -o $outf --to-tsv
    biom = pd.read_table(outf,skiprows=1,index_col=0)

    with open('picrust/%s/repseq.fasta' % marker,'wt') as hndl:
        for ind in biom.index:
            r = repseqs[ind]
            hndl.write('>'+r.id+'\n')
            hndl.write(str(r.seq).replace('N','')+'\n')

    !rm -r $digest
    !rm $outf

/home/amir/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:29: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.


In [ ]:
for marker in ['16SrRNA','ITS1']:

    fasta = 'picrust/%s/repseq.fasta' % marker
    biom = 'picrust/%s/table.biom' % marker
    outputs = 'picrust/%s/output' % marker
    EC_no_desc = 'picrust/%s/output/EC_metagenome_out/pred_metagenome_unstrat.tsv.gz' % marker
    EC_desc = 'picrust/%s/output/EC_metagenome_out/pred_metagenome_unstrat_descrip.tsv.gz' % marker
    path_no_desc = 'picrust/%s/output/pathways_out/path_abun_unstrat.tsv.gz' % marker
    path_desc = 'picrust/%s/output/pathways_out/path_abun_unstrat_descrip.tsv.gz' % marker
    
    !picrust2_pipeline.py \
        -s $fasta \
        -i $biom \
        --remove_intermediate \
        -o $outputs -p 2

    !add_descriptions.py \
        -i $EC_no_desc \
        -m EC \
        -o $EC_desc

    !add_descriptions.py \
        -i $path_no_desc \
        -m METACYC \
        -o $path_desc